# Colombian Coffee Production Prediction

## Importing and Cleaning Data

This sript will retrieve information from various sources (Including data scraped from the Internet), join them together in a final DataFrame and save it in a .csv file that will be used in the rest of the analysis to avoid running all therse stps all over again.

This script should be executed just **one time**, to repeat the rest of the analysis, is necessary, it can just be executed starting on step 1 ([EDA](01_EDA.ipynb))

In [1]:
import pandas as pd
import numpy as np
import calendar
import datetime
import time
import patsy

In [2]:
pd.set_option('display.max_rows',50)

### Weather Data

Load Weather Data pre-gererated via Scrapy scraped from Weather Underground (www.wunderground.com) for specific Colombian Locations

In [3]:
# Load Weather Data pre-gererated via Scrapy scraped from Weather Underground (www.wunderground.com)
weather_df=pd.read_json('../csv/weather.json')
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99 entries, 0 to 98
Data columns (total 8 columns):
avg_precip      99 non-null float64
avg_tmp         99 non-null int64
avg_wind        99 non-null int64
day_humidity    99 non-null object
dew_point       99 non-null int64
month           99 non-null object
sum_precip      99 non-null float64
year            99 non-null int64
dtypes: float64(2), int64(4), object(2)
memory usage: 7.0+ KB


In [4]:
weather_df.head()

,avg_precip,avg_tmp,avg_wind,day_humidity,dew_point,month,sum_precip,year
0,0.0,73,1,"[63, 67, 56, 64, 69, , , 80, , 73, 86, , 69, 6...",64,January,0.0,2010
1,0.0,76,1,"[58, 62, 59, 94, , , , 70, 87, 73, 60, 66, 66,...",64,February,0.0,2010
2,0.0,75,1,"[, , , 76, 65, 65, 62, 68, 69, , 66, 64, 69, 5...",65,March,0.0,2010
3,0.0,73,1,"[71, 71, 75, 85, 83, 90, , , , , 100, 78, 65, ...",66,April,0.0,2010
4,0.0,74,1,"[83, 76, 89, 78, 75, 80, 68, 77, 71, 75, 75, 7...",66,May,0.0,2010


**Cleaning Data (Humidity)**

Remove empty entries for daily humidity data, and make a column with the average value

In [5]:
# Make daily humility a list of numbers without blanks and get the average per month
weather_df['day_humidity'] = [list(map(int,list(filter(None,x)))) for x in weather_df['day_humidity']]
weather_df['avg_humidity'] = [np.mean(x) for x in weather_df['day_humidity']]
weather_df.head()

,avg_precip,avg_tmp,avg_wind,day_humidity,dew_point,month,sum_precip,year,avg_humidity
0,0.0,73,1,"[63, 67, 56, 64, 69, 80, 73, 86, 69, 65, 67, 6...",64,January,0.0,2010,69.625000
1,0.0,76,1,"[58, 62, 59, 94, 70, 87, 73, 60, 66, 66, 68, 8...",64,February,0.0,2010,70.153846
2,0.0,75,1,"[76, 65, 65, 62, 68, 69, 66, 64, 69, 57, 60, 6...",65,March,0.0,2010,68.500000
3,0.0,73,1,"[71, 71, 75, 85, 83, 90, 100, 78, 65, 69, 93, ...",66,April,0.0,2010,78.375000
4,0.0,74,1,"[83, 76, 89, 78, 75, 80, 68, 77, 71, 75, 75, 7...",66,May,0.0,2010,76.703704


### Coffee Data

Load Coffee Historical Data via National Coffee Association (https://www.federaciondecafeteros.org)

**Monthly production** 

This will be the target variable! (Thousand bags of coffee (60kg each)

In [6]:
# Load Coffee Historical Data via National Coffee Association (https://www.federaciondecafeteros.org)
production_df=pd.read_excel('../csv/Produccion_volumen_mensual.xls', header=5, usecols="B:C")
production_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 747 entries, 0 to 746
Data columns (total 2 columns):
Mes           747 non-null datetime64[ns]
Producción    747 non-null float64
dtypes: datetime64[ns](1), float64(1)
memory usage: 11.8 KB


**Cleaning Data (Monthly Production)**

New columns to match year/monthly basis and remove special characters from column names

In [7]:
# Create new columns year and month(1,12) to match weather information
production_df['year'] = [x.year for x in production_df['Mes']]
production_df['month'] = [calendar.month_name[x.month] for x in production_df['Mes']]

#Rename columns with special characters
production_df.rename(columns={'Producción': 'Produccion'}, inplace=True)
production_df.head()

,Mes,Produccion,year,month
0,1956-01-01,693.0,1956,January
1,1956-02-01,461.0,1956,February
2,1956-03-01,339.0,1956,March
3,1956-04-01,417.0,1956,April
4,1956-05-01,644.0,1956,May


**Historical Coffee exporting prices**

In [8]:
# Historical Coffee exporting prices
prices_df=pd.read_excel('../csv/Precios_OIC_mensual2.xls', header=8, usecols="B:B,F:F,I:I,L:L")
prices_df.columns

Index(['Mes', 'Promedio ponderado', 'Promedio ponderado.1',
       'Promedio ponderado.2'],
      dtype='object')

**Cleaning Data (Exporting Prices)**

New columns to match year/monthly basis and rename identical column names

In [9]:
# Create new columns year and month(1,12) to match weather information and id equal named columns
prices_df['year'] = [x.year for x in prices_df['Mes']]
prices_df['month'] = [calendar.month_name[x.month] for x in prices_df['Mes']]
prices_df.rename(columns={'Promedio ponderado': 'colombia_price','Promedio ponderado.1': 'others_price', 'Promedio ponderado.2': 'brasil_price'}, inplace=True)
prices_df.head()

,Mes,colombia_price,others_price,brasil_price,year,month
0,2000-01-01,130.13,111.11,97.68,2000,January
1,2000-02-01,124.73,103.44,91.51,2000,February
2,2000-03-01,119.51,100.73,89.93,2000,March
3,2000-04-01,112.67,94.61,86.46,2000,April
4,2000-05-01,110.31,94.17,87.23,2000,May


**Historical Coffee exportation Volume**

In [10]:
# Historical Coffee exporting volume
export_df=pd.read_excel('../csv/Exportaciones_volumen_mensual.xls', header=7, usecols="B:C")
export_df.head()

,MES,Total Exportaciones
0,1958-01-01,337.0
1,1958-02-01,450.0
2,1958-03-01,383.0
3,1958-04-01,334.0
4,1958-05-01,331.0


**Cleanig Data (Exportation Volume)**

Create new columns to match year/monthly basis and rename columns with significant names

In [11]:
# Create new columns year and month(1,12) to match weather information
export_df['year'] = [x.year for x in export_df['MES']]
export_df['month'] = [calendar.month_name[x.month] for x in export_df['MES']]
export_df.rename(columns={'Total Exportaciones': 'vol_export'}, inplace=True)
export_df.head()

,MES,vol_export,year,month
0,1958-01-01,337.0,1958,January
1,1958-02-01,450.0,1958,February
2,1958-03-01,383.0,1958,March
3,1958-04-01,334.0,1958,April
4,1958-05-01,331.0,1958,May


**Historical Rate conversion Dollar vs Colombia Peso**

In [12]:
# Historical Rate conversion Dollar vs Colombia Peso
dollar_df=pd.read_excel('../csv/historia.xls', header=0, usecols="A:C")
dollar_df.head()

,Fecha,Festivo,TCRM
0,1991-12-01,NaN,643.42
1,1991-12-02,NaN,643.42
2,1991-12-03,NaN,639.22
3,1991-12-04,NaN,635.70
4,1991-12-05,NaN,631.51


**Data Cleaning (Dollar Rate - TRM)**

Create new columns to match year/monthly basis and rename columns with significant names

In [13]:
# Create new columns year and month(1,12) to match weather information and
dollar_df['year'] = [x.year for x in dollar_df['Fecha']]
dollar_df['month'] = [calendar.month_name[x.month] for x in dollar_df['Fecha']]
dollar_df.rename(columns={'TCRM': 'trm'}, inplace=True)

Create column with monthly average value

In [14]:
# Dollar rate values are daily: Take average per month
dollar_df = dollar_df.groupby(['year', 'month'])['trm'].mean().reset_index()
dollar_df = dollar_df.dropna()
dollar_df.tail()

,year,month,trm
312,2017,September,2917.085000
313,2018,April,2753.902000
316,2018,February,2860.252857
317,2018,January,2868.572258
320,2018,March,2847.930645


**Production information on Top 10 countries - By year**

In [15]:
# Yearly production information on Top 10 countries - New Model
country_df=pd.read_excel('../csv/1a - Total production.xlsx', header=3, usecols="A:AD")
country_df.head()

,Crop year,Unnamed: 1,1990/91,1991/92,1992/93,1993/94,1994/95,1995/96,1996/97,1997/98,...,2008/09,2009/10,2010/11,2011/12,2012/13,2013/14,2014/15,2015/16,2016/17,2017/18
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,April group,NaN,40795.2926,42532.5454,46870.7042,40176.7776,39995.291,29402.0492,44301.040,39462.377,...,67566.9293,61582.4778,69824.7609,70160.8434,74358.9813,73107.2010,69345.9090,68600.5490,73664.5366,69804.0
2,Angola,(R/A),50.3450,79.3310,77.5200,32.6080,76.802,62.1090,70.925,64.330,...,37.6100,13.4200,34.9700,28.7150,32.7900,34.9350,39.4050,40.5150,44.8300,40.0
3,Bolivia,(A),122.7770,106.5360,122.7350,57.3230,122.444,151.9850,133.079,153.219,...,135.4186,142.4751,129.7249,142.8354,114.7812,127.9122,106.3766,89.2191,80.9835,86.0
4,Brazil,(A/R),27285.6286,27293.4934,34603.3542,28166.9786,28192.047,18060.2022,29196.743,26148.004,...,50490.0837,43976.8120,53428.4102,50591.8289,55420.2250,54698.0833,52298.9787,50387.6396,54999.7634,51000.0


**Cleaning Data (Other Countries Information)**

Drop unused columns and rename columns to remove special characters

In [16]:
# Clean Data: Remove Nans, and rename columns
country_df = country_df.dropna()
country_df = country_df.drop('Unnamed: 1', axis=1)
country_df.rename(columns={ '1990/91': '1990', '1991/92': '1991', '1992/93': '1992', '1993/94': '1993'
                           , '1994/95': '1994', '1995/96': '1995', '1996/97': '1996', '1997/98': '1997'
                           , '1998/99': '1998', '1999/00': '1999', '2000/01': '2000', '2001/02': '2001'
                           , '2002/03': '2002', '2003/04': '2003', '2004/05': '2004', '2005/06': '2005'
                           , '2006/07': '2006', '2007/08': '2007', '2008/09': '2008', '2009/10': '2009'
                           , '2010/11': '2010', '2011/12': '2011', '2012/13': '2012', '2013/14': '2013'
                           , '2014/15': '2014', '2015/16': '2015', '2016/17': '2016', '2017/18': '2017'}, inplace=True)
country_df.head()

,Crop year,1990,1991,1992,1993,1994,1995,1996,1997,1998,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
2,Angola,50.3450,79.3310,77.5200,32.6080,76.802,62.1090,70.925,64.330,85.3440,...,37.6100,13.4200,34.9700,28.7150,32.7900,34.9350,39.4050,40.5150,44.8300,40.0
3,Bolivia,122.7770,106.5360,122.7350,57.3230,122.444,151.9850,133.079,153.219,149.9850,...,135.4186,142.4751,129.7249,142.8354,114.7812,127.9122,106.3766,89.2191,80.9835,86.0
4,Brazil,27285.6286,27293.4934,34603.3542,28166.9786,28192.047,18060.2022,29196.743,26148.004,36760.8533,...,50490.0837,43976.8120,53428.4102,50591.8289,55420.2250,54698.0833,52298.9787,50387.6396,54999.7634,51000.0
5,Burundi,487.3930,667.1990,620.2380,393.3540,664.143,433.9800,400.969,249.785,491.9920,...,411.7822,111.6130,352.9776,204.1328,405.9615,163.2177,247.5500,274.1017,258.2203,250.0
6,Ecuador,1503.8150,2123.8240,1185.4800,2069.0070,2375.766,1888.2330,1992.914,1190.663,1205.9680,...,770.6473,813.2849,853.9798,825.4144,828.1024,665.5450,644.0112,644.4926,644.8845,625.0


Use only top 10 countries and transpose DataFrame (Country names as columns)

In [17]:
#Use only top 10 countries and transpose DataFrame (Country names as columns)
top10_df = country_df.sort_values('2017', ascending=False).head(10)
top10_df = top10_df.set_index('Crop year').T
top10_df

Crop year,Brazil,Vietnam,Colombia,Indonesia,Honduras,Ethiopia,India,Uganda,Peru,Mexico
1990,27285.6286,1310.2880,14268.3870,7441.3830,1567.9580,2909.4510,2828.6870,1955.009,936.6320,4674.2450
1991,27293.4934,1437.8480,17814.8660,8493.1960,2321.8080,2924.6640,2999.7490,2088.001,1200.4740,4727.2060
1992,34603.3542,2340.4470,15189.1310,5569.4780,1918.2090,1825.1710,2823.4970,2185.074,1761.9530,3400.8110
1993,28166.9786,3020.2160,10297.6020,6743.2880,1829.1790,2967.0040,3533.1810,3141.706,665.4970,5023.4880
1994,28192.0470,3631.6090,12862.4700,5367.8780,2180.7300,2762.7300,3001.8970,2392.753,1178.6280,4158.7770
1995,18060.2022,3970.0860,12962.5610,4573.4290,1909.2380,3165.3220,3716.9990,3243.584,1870.9390,5299.8880
1996,29196.7430,5752.4810,10719.0630,8220.5840,2004.3880,3407.4420,3417.4540,3418.724,1805.8610,5109.5510
1997,26148.0040,6978.9420,12219.4550,7620.7780,2564.4050,3056.8640,3805.0890,3439.868,1929.9590,4801.5610
1998,36760.8533,7051.7220,10979.0940,7295.9550,2195.1450,2927.7250,4416.7680,3385.839,2102.2980,4801.3480
1999,47577.8065,11630.9138,9678.7709,6557.0662,2985.0050,3784.0510,4866.5892,2862.447,2743.5330,6218.6765


Include one column with total by year

In [18]:
top10_df['total'] = top10_df['Brazil'] + top10_df['Vietnam'] + top10_df['Colombia'] + top10_df['Indonesia'] + top10_df['Honduras'] + top10_df['Ethiopia'] + top10_df['India'] + top10_df['Uganda'] + top10_df['Peru'] + top10_df['Mexico']
top10_df.head()

Crop year,Brazil,Vietnam,Colombia,Indonesia,Honduras,Ethiopia,India,Uganda,Peru,Mexico,total
1990,27285.6286,1310.288,14268.387,7441.383,1567.958,2909.451,2828.687,1955.009,936.632,4674.245,65177.6686
1991,27293.4934,1437.848,17814.866,8493.196,2321.808,2924.664,2999.749,2088.001,1200.474,4727.206,71301.3054
1992,34603.3542,2340.447,15189.131,5569.478,1918.209,1825.171,2823.497,2185.074,1761.953,3400.811,71617.1252
1993,28166.9786,3020.216,10297.602,6743.288,1829.179,2967.004,3533.181,3141.706,665.497,5023.488,65388.1396
1994,28192.0470,3631.609,12862.470,5367.878,2180.730,2762.730,3001.897,2392.753,1178.628,4158.777,65729.5190


## Join Information together

### Monthly Data

- Production Data (Target Variable) since 1956

In [19]:
len(production_df)

747

- Weather Data since 2010

In [20]:
# Join Production (Target) to weather data
merge_df = pd.merge(production_df, weather_df, how='inner', on=['year', 'month'])
len(merge_df)

99

- Exporting Prices Data since 1956 (Will match only data since 2010)

In [21]:
# Join to prices data
merge_df = pd.merge(merge_df, prices_df, how='inner', on=['year', 'month'])
len(merge_df)

99

- Exporting Volumes Data since 1956 (Will match only data since 2010 and does not have data for april 2018)

In [22]:
# Join to exportation volume data
merge_df = pd.merge(merge_df, export_df, how='inner', on=['year', 'month'])
len(merge_df)

98

- Dollar Rate Data since 1956 (Will match only data since 2010)

In [23]:
# Join to dollar rate data
merge_df = pd.merge(merge_df, dollar_df, how='inner', on=['year', 'month'])
len(merge_df)

98

In [24]:
merge_df.columns

Index(['Mes_x', 'Produccion', 'year', 'month', 'avg_precip', 'avg_tmp',
       'avg_wind', 'day_humidity', 'dew_point', 'sum_precip', 'avg_humidity',
       'Mes_y', 'colombia_price', 'others_price', 'brasil_price', 'MES',
       'vol_export', 'trm'],
      dtype='object')

- Create and join categorical Variable from Month Column

In [25]:
month_df=patsy.dmatrix('month',data=merge_df,return_type='dataframe')
month_df.head()

,Intercept,month[T.August],month[T.December],month[T.February],month[T.January],month[T.July],month[T.June],month[T.March],month[T.May],month[T.November],month[T.October],month[T.September]
0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [26]:
merge_df=merge_df.join(month_df)
merge_df.head()

,Mes_x,Produccion,year,month,avg_precip,avg_tmp,avg_wind,day_humidity,dew_point,sum_precip,...,month[T.December],month[T.February],month[T.January],month[T.July],month[T.June],month[T.March],month[T.May],month[T.November],month[T.October],month[T.September]
0,2010-01-01,515.0,2010,January,0.0,73,1,"[63, 67, 56, 64, 69, 80, 73, 86, 69, 65, 67, 6...",64,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2010-02-01,648.0,2010,February,0.0,76,1,"[58, 62, 59, 94, 70, 87, 73, 60, 66, 66, 68, 8...",64,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2010-03-01,629.0,2010,March,0.0,75,1,"[76, 65, 65, 62, 68, 69, 66, 64, 69, 57, 60, 6...",65,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,2010-04-01,647.0,2010,April,0.0,73,1,"[71, 71, 75, 85, 83, 90, 100, 78, 65, 69, 93, ...",66,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2010-05-01,822.0,2010,May,0.0,74,1,"[83, 76, 89, 78, 75, 80, 68, 77, 71, 75, 75, 7...",66,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


- Create and Save final DataFrame

In [27]:
merge_df.columns

Index(['Mes_x', 'Produccion', 'year', 'month', 'avg_precip', 'avg_tmp',
       'avg_wind', 'day_humidity', 'dew_point', 'sum_precip', 'avg_humidity',
       'Mes_y', 'colombia_price', 'others_price', 'brasil_price', 'MES',
       'vol_export', 'trm', 'Intercept', 'month[T.August]',
       'month[T.December]', 'month[T.February]', 'month[T.January]',
       'month[T.July]', 'month[T.June]', 'month[T.March]', 'month[T.May]',
       'month[T.November]', 'month[T.October]', 'month[T.September]'],
      dtype='object')

In [28]:
# Generate and save final DataFrame
merge_df = merge_df[['Produccion', 'year', 'month', 'avg_precip', 'avg_tmp', 
                     'avg_wind', 'dew_point', 'avg_humidity', 'colombia_price', 'brasil_price',
                     'others_price', 'vol_export', 'trm', 'Intercept', 'month[T.January]', 
                     'month[T.February]', 'month[T.March]', 'month[T.May]', 'month[T.June]', 
                     'month[T.July]', 'month[T.August]', 'month[T.September]', 'month[T.October]', 
                     'month[T.November]', 'month[T.December]']]
merge_df.to_csv('../csv/FINAL_coffee_mth.csv')

## Yearly Data

In [29]:
top10_df.to_csv('../csv/FINAL_coffee_yr.csv')